In [2]:
import json
import math
import copy
import os
import time
import numpy as np
from pathlib import Path
import networkx as nx
import osmnx as ox
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as geopd
import shapely
from shapely import ops
import pyproj
import requests
from itertools import chain
from modules import utils
from networkx import bfs_edges

# multiprocessing for faster download
import ray
ray.init()

import modules.network_extractor as net_ex
import modules.networker as netw

2025-03-07 17:24:23,982	INFO worker.py:1832 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8266 


In [3]:
import modules.network_extractor as net_ex
from modules.networker import Networker
extractor = net_ex.NetworkExtractor()
extractor.DATA_BASE_PATH = "/home/geolab/Desktop/Research/data"
networker = Networker()

folder = "bologna"

In [4]:
import warnings
warnings.filterwarnings("ignore", category=np.RankWarning) 

extractor = net_ex.NetworkExtractor()
extractor.DATA_BASE_PATH = "/home/geolab/Desktop/Research/data"
networker = netw.Networker()

In [3]:
urban_centers: geopd.GeoDataFrame

urban_centers = geopd.read_file(
    "/home/geolab/Desktop/Research/data/GHS_URBAN_CENTERS/GHS_URBAN_SIMPLIFIED_fixed.gpkg",
    layer='GHS_URBAN_SIMPLIFIED_fixed'    
)
urban_centers = urban_centers.rename(columns={
    "GC_POP_TOT_2025": "population",
    "GC_UCA_KM2_2025": "area",
    "GC_DEV_USR_2025": "continent",
    "GC_UCN_MAI_2025": "name",
    "GC_CNT_GAD_2025": "country"
})
transform = pyproj.Transformer.from_crs("ESRI:54009", "EPSG:4326", always_xy=True).transform

cities = {
    "ottawa": {"search": "Kampala", "country": "Uganda"},
}

# Extract the networks for the cities and save statistics about each one
for folder, city in cities.items():
    #try:
    city_info = urban_centers.loc[
        (urban_centers["name"] == city["search"]) &
        (urban_centers["country"] == city["country"])
    ]

    geometry = city_info["geometry"].values[0]
    geom_reprojected = ops.transform(transform, geometry)
    print(geom_reprojected)

MULTIPOLYGON (((32.518271512022615 0.4771832666409849, 32.528239521618346 0.4771832666409849, 32.52824260383152 0.4771828707965358, 32.52824538202709 0.4771817220112015, 32.528247584255915 0.4771799327360949, 32.52824899494853 0.4771776781179297, 32.528249476016505 0.4771751788544413, 32.528226109918755 0.4690954819069344, 32.55815007316406 0.4690954819069344, 32.55815315539355 0.4690950860626623, 32.558155933640876 0.4690939372778412, 32.55815813595186 0.4690921480035335, 32.558159546748975 0.4690898933863754, 32.55816002793357 0.4690873941240032, 32.55813704016707 0.4610077007542762, 32.608006951002764 0.4610077007542762, 32.60802997398006 0.4690873941240032, 32.608030469520685 0.4690898933863754, 32.60803189326861 0.4690921480035335, 32.60803410585747 0.4690939372778412, 32.60803689070362 0.4690950860626623, 32.608039975206914 0.4690954819069344, 32.61800797763507 0.4690954819069344, 32.618031408226095 0.4771751788544413, 32.61803190389318 0.4771776781179297, 32.618033327755896 0.47

In [8]:
@ray.remote
def download_network(network_type:str, place, geom_reprojected, city_name) -> nx.Graph:
    # Extract driving graph and save statistics
    print(network_type)
    city_drive = extractor.extract_network(network_type, place, geometry=geom_reprojected)
    networker.add_elevation(city_drive, city_name=city_name, cpus=4)
    #add_edge_slope(city_drive)
    ox.add_edge_grades(city_drive)
    ox.add_edge_bearings(city_drive)
    return city_drive

In [10]:
cities = {
    "bologna": {"search": "Bologna", "country": "Italy"},
}

stats_dict = {
    "city": [],
    "area": [],
    "population": [],
    "drive_nodes": [],
    "drive_edges": [],
    "bike_nodes": [],
    "bike_edges": [],
    "walk_nodes": [],
    "walk_edges": [],
    "total_time": []
}

def add_stat(stats, stat, value):
    stats[stat].append(value)

# Extract the networks for the cities and save statistics about each one
for folder, city in cities.items():
    #try:
    city_info = urban_centers.loc[
        (urban_centers["name"] == city["search"]) &
        (urban_centers["country"] == city["country"])
    ]

    geometry = city_info["geometry"].values[0]
    geom_reprojected = ops.transform(transform, geometry)

    add_stat(stats_dict, "area", city_info["area"].values[0])
    add_stat(stats_dict, "population", int(city_info["population"].values[0]))
    print(f"Total Area {city_info['area'].values[0]}")

    # start timer to check time to download graphs for the city.
    start = time.time()
    city_name = city["search"]
    add_stat(stats_dict, "city", city_name)
    print(f"extracting {city_name}")
    place = city_name
    stat_row = {}

    networks = ["drive", "bike", "walk"]

    network_futures = {}
    for network in networks:
        network_futures[network] = download_network.remote(network, place, geom_reprojected, city_name)
    
    print(network_futures)
    city_drive = ray.get(network_futures["drive"])
    city_bike = ray.get(network_futures["bike"])
    city_walk = ray.get(network_futures["walk"])



    # # Extract driving graph and save statistics
    # city_drive = extractor.extract_network("drive", place, geometry=geom_reprojected)
    # networker.add_elevation(city_drive, city_name=city_name, cpus=4)
    # ox.add_edge_grades(city_drive)
    # ox.add_edge_bearings(city_drive)
    add_stat(stats_dict, "drive_nodes", city_drive.number_of_nodes())
    add_stat(stats_dict, "drive_edges", city_drive.number_of_edges())
    
    # Extract bike graph and save statistics
    # city_bike = extractor.extract_network("bike", place, geometry=geom_reprojected)
    # networker.add_elevation(city_bike, city_name=city_name, cpus=4)
    # ox.add_edge_grades(city_bike)
    # ox.add_edge_bearings(city_bike)
    add_stat(stats_dict, "bike_nodes", city_bike.number_of_nodes())
    add_stat(stats_dict, "bike_edges", city_bike.number_of_edges())

    # Extract pedestrian graph and save statistics
    # city_walk = extractor.extract_network("walk", place, geometry=geom_reprojected)
    # networker.add_elevation(city_walk, city_name=city_name, cpus=4)
    # ox.add_edge_grades(city_walk)
    # ox.add_edge_bearings(city_walk)
    add_stat(stats_dict, "walk_nodes", city_walk.number_of_nodes())
    add_stat(stats_dict, "walk_edges", city_walk.number_of_edges())

    print("Saving the city graphs")
    # Saving the city graphs
    base_folder = f"{extractor.DATA_BASE_PATH}/{folder}/graph"
    Path(base_folder).mkdir(parents=True, exist_ok=True)
    extractor.save_as_graph(city_drive, f"{folder}/graph/{folder}_drive")
    extractor.save_as_graph(city_bike, f"{folder}/graph/{folder}_bike")
    extractor.save_as_graph(city_walk, f"{folder}/graph/{folder}_walk")

    end = time.time()
    total_time = end-start
    stat_row["total_time"] = total_time

    add_stat(stats_dict, "total_time", total_time)
    print(f"{city_name} Finished!")
    print("------------------------------------------------------------")

    # except Exception as ex:
    #     print(ex.with_traceback())
    #     print(f"error in city {city}")

# stats = pd.DataFrame(stats_dict)
# stats.to_csv(f"{extractor.DATA_BASE_PATH}/stats.csv")
stats_dict

Total Area 114
extracting Bologna
{'drive': ObjectRef(c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000), 'bike': ObjectRef(16310a0f0a45af5cffffffffffffffffffffffff0100000001000000), 'walk': ObjectRef(c2668a65bda616c1ffffffffffffffffffffffff0100000001000000)}
Saving the city graphs
Bologna Finished!
------------------------------------------------------------


(download_network pid=2864531) walk
(download_network pid=2864524) drive
(download_network pid=2864525) bike


(download_network pid=2864531) /home/geolab/Desktop/Research/notebooks/modules/utils.py:30: RankWarning: Polyfit may be poorly conditioned
(download_network pid=2864531)   m,b = np.polyfit(x,y,1)
(download_network pid=2864531) /home/geolab/Desktop/Research/notebooks/modules/utils.py:30: RankWarning: Polyfit may be poorly conditioned
(download_network pid=2864531)   m,b = np.polyfit(x,y,1)
(download_network pid=2864531) /home/geolab/Desktop/Research/notebooks/modules/utils.py:30: RankWarning: Polyfit may be poorly conditioned
(download_network pid=2864531)   m,b = np.polyfit(x,y,1)
(download_network pid=2864531) /home/geolab/Desktop/Research/notebooks/modules/utils.py:30: RankWarning: Polyfit may be poorly conditioned
(download_network pid=2864531)   m,b = np.polyfit(x,y,1)
(download_network pid=2864531) /home/geolab/Desktop/Research/notebooks/modules/utils.py:30: RankWarning: Polyfit may be poorly conditioned
(download_network pid=2864531)   m,b = np.polyfit(x,y,1)
(download_network pi

(download_network pid=2864531) 0
(download_network pid=2864531) 1000
(download_network pid=2864531) 2000
(download_network pid=2864531) 3000
(download_network pid=2864531) 4000
(download_network pid=2864531) 5000
(download_network pid=2864531) 6000
(download_network pid=2864531) 7000
(download_network pid=2864531) 8000
(download_network pid=2864531) 9000
(download_network pid=2864531) 10000
(download_network pid=2864531) 11000
(download_network pid=2864531) 12000
(download_network pid=2864531) 13000
(download_network pid=2864531) 14000
(download_network pid=2864531) 15000
(download_network pid=2864531) 16000
(download_network pid=2864531) 17000
(download_network pid=2864531) 18000
(download_network pid=2864531) 19000
(download_network pid=2864531) 20000
(download_network pid=2864531) 21000
(download_network pid=2864531) 22000
(download_network pid=2864531) 23000
(download_network pid=2864531) 24000
(download_network pid=2864531) 25000
(download_network pid=2864531) 26000
(download_netw

{'city': ['Bologna'],
 'area': [114],
 'population': [456780],
 'drive_nodes': [7288],
 'drive_edges': [14485],
 'bike_nodes': [18725],
 'bike_edges': [21577],
 'walk_nodes': [28507],
 'walk_edges': [36236],
 'total_time': [115.46182298660278]}

In [1]:
# Example of loading a graph after downloading
folder = "bologna"
g_walk = extractor.load_graph(f"{folder}/graph/{folder}_walk")

NameError: name 'extractor' is not defined

In [ ]:
g_walk.number_of_edges()

36236